1. I need to do is find which model would be best for our circumstances (multi model output, binary classifier any others?)  

2. just chuck the SDQ, APQ, and FCM into a model (kinda like a base model???) and run it
while running figure out
3. for FCM, which “square”s has most substantial info to help us determine ADHD diganosis  
4. top SDQ for gender prediction   
5. top APQ for gender prediction  
6. chuck 3-5 into model

notes:
* may not need to do one hot encoding because catergorical data might not be helpful in this model


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import seaborn as sns

In [4]:
file_path_trainQ = "../../data/raw/train/TRAIN_QUANTITATIVE.csv"
train_Quant = pd.read_csv(file_path_trainQ)
# train_Quant

In [ ]:
file_path_trainFM = "../../data/raw/train/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv"
train_FM = pd.read_csv(file_path_trainFM)
train_FM.head()

,participant_id,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
0,70z8Q2xdTXM3,0.222930,0.527903,0.429966,0.060457,0.566489,0.315342,0.508408,-0.078290,0.525692,...,0.224985,0.397448,0.422966,0.184642,0.305549,0.420349,0.016328,0.561864,0.471170,0.365221
1,WHWymJu6zNZi,0.614765,0.577255,0.496127,0.496606,0.404686,0.439724,0.122590,-0.085452,0.120673,...,0.217546,-0.014549,0.000440,-0.096451,0.454501,0.343916,0.167313,0.607656,0.550623,0.503176
2,4PAQp1M6EyAo,-0.116833,0.458408,0.260703,0.639031,0.769337,0.442528,0.637110,0.192010,0.520379,...,0.342487,-0.021141,-0.037836,0.075069,0.412712,0.292708,0.391005,0.461544,0.508912,0.624232
3,obEacy4Of68I,0.199688,0.752714,0.658283,0.575096,0.692867,0.645789,0.522750,0.412188,0.530843,...,0.103562,-0.178313,0.210983,-0.018666,0.436313,0.592982,0.216205,0.341272,0.440313,0.558193
4,s7WzzDcmDOhF,0.227321,0.613268,0.621447,0.562673,0.736709,0.589813,0.266676,0.359668,0.300771,...,-0.164956,0.007064,-0.120904,-0.488095,0.493575,-0.215361,0.210685,0.055850,0.119065,0.108273


## Preparing X_Train / Features

In [6]:
train_Quant.isnull().sum()

participant_id                  0
EHQ_EHQ_Total                  13
ColorVision_CV_Score           23
APQ_P_APQ_P_CP                 12
APQ_P_APQ_P_ID                 12
APQ_P_APQ_P_INV                12
APQ_P_APQ_P_OPD                12
APQ_P_APQ_P_PM                 12
APQ_P_APQ_P_PP                 12
SDQ_SDQ_Conduct_Problems        9
SDQ_SDQ_Difficulties_Total      9
SDQ_SDQ_Emotional_Problems      9
SDQ_SDQ_Externalizing           9
SDQ_SDQ_Generating_Impact       9
SDQ_SDQ_Hyperactivity           9
SDQ_SDQ_Internalizing           9
SDQ_SDQ_Peer_Problems           9
SDQ_SDQ_Prosocial               9
MRI_Track_Age_at_Scan         360
dtype: int64

In [7]:
train_Quant = train_Quant.drop(columns=['MRI_Track_Age_at_Scan'])

In [8]:
train_Quant.shape

(1213, 18)

In [9]:
train_Quant_nulldrop = train_Quant.dropna()
train_Quant_nulldrop.shape

(1159, 18)

In [10]:
# train_Quant_nulldrop

In [11]:
rows_dropped = train_Quant.shape[0] - train_Quant_nulldrop.shape[0]
rows_dropped

54

In [12]:
rows_dropped / train_Quant.shape[0] * 100 # percentage of rows dropped

4.451772464962902

In [13]:
X_train_ppl = pd.merge(train_FM, train_Quant_nulldrop, on = 'participant_id')#.drop(columns=['participant_id'])

# ensure it looks accurate
X_train_ppl

,participant_id,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,...,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial
0,70z8Q2xdTXM3,0.222930,0.527903,0.429966,0.060457,0.566489,0.315342,0.508408,-0.078290,0.525692,...,29.0,0.0,8.0,1.0,4.0,5.0,4.0,4.0,3.0,9.0
1,4PAQp1M6EyAo,-0.116833,0.458408,0.260703,0.639031,0.769337,0.442528,0.637110,0.192010,0.520379,...,29.0,8.0,26.0,4.0,17.0,8.0,9.0,9.0,5.0,8.0
2,obEacy4Of68I,0.199688,0.752714,0.658283,0.575096,0.692867,0.645789,0.522750,0.412188,0.530843,...,24.0,0.0,7.0,2.0,4.0,1.0,4.0,3.0,1.0,10.0
3,s7WzzDcmDOhF,0.227321,0.613268,0.621447,0.562673,0.736709,0.589813,0.266676,0.359668,0.300771,...,23.0,3.0,14.0,3.0,6.0,3.0,3.0,8.0,5.0,6.0
4,s55ZRHrJtnlL,0.337316,0.823123,0.733585,0.418883,0.771445,0.653699,0.693189,0.345408,0.729822,...,18.0,4.0,17.0,4.0,13.0,4.0,9.0,4.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,9gpepMI9sj5q,0.265284,0.551726,0.628318,0.647700,0.710757,0.132334,0.326207,0.354319,0.531170,...,20.0,0.0,5.0,3.0,2.0,0.0,2.0,3.0,0.0,8.0
1155,FIDen5rdMc0v,-0.018377,0.576689,0.527451,0.327463,0.586868,0.573689,0.300544,0.301103,0.659840,...,29.0,3.0,16.0,1.0,12.0,5.0,9.0,4.0,3.0,8.0
1156,dlsMC4TXL4e8,0.227028,0.405659,0.023545,-0.093085,-0.068960,0.647574,0.762552,0.465109,0.199337,...,25.0,2.0,14.0,2.0,12.0,5.0,10.0,2.0,0.0,8.0
1157,syeyZjEx8FUx,0.189849,0.752876,0.842463,0.817037,0.820196,0.792950,0.650929,0.488504,0.580207,...,21.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,9.0


In [ ]:
# train_FM

,participant_id,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
0,70z8Q2xdTXM3,0.222930,0.527903,0.429966,0.060457,0.566489,0.315342,0.508408,-0.078290,0.525692,...,0.224985,0.397448,0.422966,0.184642,0.305549,0.420349,0.016328,0.561864,0.471170,0.365221
1,WHWymJu6zNZi,0.614765,0.577255,0.496127,0.496606,0.404686,0.439724,0.122590,-0.085452,0.120673,...,0.217546,-0.014549,0.000440,-0.096451,0.454501,0.343916,0.167313,0.607656,0.550623,0.503176
2,4PAQp1M6EyAo,-0.116833,0.458408,0.260703,0.639031,0.769337,0.442528,0.637110,0.192010,0.520379,...,0.342487,-0.021141,-0.037836,0.075069,0.412712,0.292708,0.391005,0.461544,0.508912,0.624232
3,obEacy4Of68I,0.199688,0.752714,0.658283,0.575096,0.692867,0.645789,0.522750,0.412188,0.530843,...,0.103562,-0.178313,0.210983,-0.018666,0.436313,0.592982,0.216205,0.341272,0.440313,0.558193
4,s7WzzDcmDOhF,0.227321,0.613268,0.621447,0.562673,0.736709,0.589813,0.266676,0.359668,0.300771,...,-0.164956,0.007064,-0.120904,-0.488095,0.493575,-0.215361,0.210685,0.055850,0.119065,0.108273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,9gpepMI9sj5q,0.265284,0.551726,0.628318,0.647700,0.710757,0.132334,0.326207,0.354319,0.531170,...,0.127097,0.129787,0.298987,0.114577,0.534109,0.118893,0.181292,0.181055,0.238357,0.577009
1209,FIDen5rdMc0v,-0.018377,0.576689,0.527451,0.327463,0.586868,0.573689,0.300544,0.301103,0.659840,...,0.482214,-0.159587,-0.162498,-0.093249,0.309914,0.143818,0.218337,0.389331,0.328741,0.238443
1210,dlsMC4TXL4e8,0.227028,0.405659,0.023545,-0.093085,-0.068960,0.647574,0.762552,0.465109,0.199337,...,-0.379304,0.126976,0.193695,0.214483,0.429836,0.302141,0.104774,0.856375,0.303248,0.363639
1211,syeyZjEx8FUx,0.189849,0.752876,0.842463,0.817037,0.820196,0.792950,0.650929,0.488504,0.580207,...,-0.212254,-0.055539,-0.221935,-0.202464,0.602651,0.482066,0.399363,0.373746,0.279900,0.684434


## Moving onto preparing Y_Train

In [14]:
file_path_targets = "../../data/raw/train/TRAINING_SOLUTIONS.csv"
Y_train_raw = pd.read_csv(file_path_targets)
Y_train_raw.head()

,participant_id,ADHD_Outcome,Sex_F
0,UmrK0vMLopoR,1,1
1,CPaeQkhcjg7d,1,0
2,Nb4EetVPm3gs,1,0
3,p4vPhVu91o4b,1,1
4,M09PXs7arQ5E,1,1


In [ ]:
Y_train_match = pd.merge(X_train_ppl, Y_train_raw, on = 'participant_id')#.drop(columns=['participant_id'])
Y_train_match

,participant_id,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,...,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,ADHD_Outcome,Sex_F
0,70z8Q2xdTXM3,0.222930,0.527903,0.429966,0.060457,0.566489,0.315342,0.508408,-0.078290,0.525692,...,8.0,1.0,4.0,5.0,4.0,4.0,3.0,9.0,1,0
1,4PAQp1M6EyAo,-0.116833,0.458408,0.260703,0.639031,0.769337,0.442528,0.637110,0.192010,0.520379,...,26.0,4.0,17.0,8.0,9.0,9.0,5.0,8.0,1,1
2,obEacy4Of68I,0.199688,0.752714,0.658283,0.575096,0.692867,0.645789,0.522750,0.412188,0.530843,...,7.0,2.0,4.0,1.0,4.0,3.0,1.0,10.0,1,1
3,s7WzzDcmDOhF,0.227321,0.613268,0.621447,0.562673,0.736709,0.589813,0.266676,0.359668,0.300771,...,14.0,3.0,6.0,3.0,3.0,8.0,5.0,6.0,1,1
4,s55ZRHrJtnlL,0.337316,0.823123,0.733585,0.418883,0.771445,0.653699,0.693189,0.345408,0.729822,...,17.0,4.0,13.0,4.0,9.0,4.0,0.0,7.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,9gpepMI9sj5q,0.265284,0.551726,0.628318,0.647700,0.710757,0.132334,0.326207,0.354319,0.531170,...,5.0,3.0,2.0,0.0,2.0,3.0,0.0,8.0,0,1
1155,FIDen5rdMc0v,-0.018377,0.576689,0.527451,0.327463,0.586868,0.573689,0.300544,0.301103,0.659840,...,16.0,1.0,12.0,5.0,9.0,4.0,3.0,8.0,1,0
1156,dlsMC4TXL4e8,0.227028,0.405659,0.023545,-0.093085,-0.068960,0.647574,0.762552,0.465109,0.199337,...,14.0,2.0,12.0,5.0,10.0,2.0,0.0,8.0,1,0
1157,syeyZjEx8FUx,0.189849,0.752876,0.842463,0.817037,0.820196,0.792950,0.650929,0.488504,0.580207,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,9.0,0,1


In [16]:
participant_id =  Y_train_match[["participant_id"]]
participant_id

,participant_id
0,70z8Q2xdTXM3
1,4PAQp1M6EyAo
2,obEacy4Of68I
3,s7WzzDcmDOhF
4,s55ZRHrJtnlL
...,...
1154,9gpepMI9sj5q
1155,FIDen5rdMc0v
1156,dlsMC4TXL4e8
1157,syeyZjEx8FUx


In [17]:
Y_train_all = Y_train_match[["ADHD_Outcome", "Sex_F"]]
Y_train_all

,ADHD_Outcome,Sex_F
0,1,0
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
1154,0,1
1155,1,0
1156,1,0
1157,0,1


In [18]:
Y_train_ADHD = Y_train_all[["ADHD_Outcome"]]
Y_train_ADHD

,ADHD_Outcome
0,1
1,1
2,1
3,1
4,1
...,...
1154,0
1155,1
1156,1
1157,0


In [19]:
Y_train_Sex = Y_train_all[["Sex_F"]]
# Y_train_Sex 

In [20]:
X_train = X_train_ppl.drop(columns=['participant_id'])

In [21]:
X_train

,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,0throw_10thcolumn,...,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial
0,0.222930,0.527903,0.429966,0.060457,0.566489,0.315342,0.508408,-0.078290,0.525692,0.470063,...,29.0,0.0,8.0,1.0,4.0,5.0,4.0,4.0,3.0,9.0
1,-0.116833,0.458408,0.260703,0.639031,0.769337,0.442528,0.637110,0.192010,0.520379,0.378557,...,29.0,8.0,26.0,4.0,17.0,8.0,9.0,9.0,5.0,8.0
2,0.199688,0.752714,0.658283,0.575096,0.692867,0.645789,0.522750,0.412188,0.530843,0.259596,...,24.0,0.0,7.0,2.0,4.0,1.0,4.0,3.0,1.0,10.0
3,0.227321,0.613268,0.621447,0.562673,0.736709,0.589813,0.266676,0.359668,0.300771,0.331445,...,23.0,3.0,14.0,3.0,6.0,3.0,3.0,8.0,5.0,6.0
4,0.337316,0.823123,0.733585,0.418883,0.771445,0.653699,0.693189,0.345408,0.729822,0.357762,...,18.0,4.0,17.0,4.0,13.0,4.0,9.0,4.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,0.265284,0.551726,0.628318,0.647700,0.710757,0.132334,0.326207,0.354319,0.531170,0.340388,...,20.0,0.0,5.0,3.0,2.0,0.0,2.0,3.0,0.0,8.0
1155,-0.018377,0.576689,0.527451,0.327463,0.586868,0.573689,0.300544,0.301103,0.659840,0.434427,...,29.0,3.0,16.0,1.0,12.0,5.0,9.0,4.0,3.0,8.0
1156,0.227028,0.405659,0.023545,-0.093085,-0.068960,0.647574,0.762552,0.465109,0.199337,-0.357953,...,25.0,2.0,14.0,2.0,12.0,5.0,10.0,2.0,0.0,8.0
1157,0.189849,0.752876,0.842463,0.817037,0.820196,0.792950,0.650929,0.488504,0.580207,0.162963,...,21.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,9.0


### Train Logistic Regression

#### Supervised ML models require dataframe with  
1. indepedent variables
2. dependent variable(s) aka target variables 
    - what you're trying to predict  

make sure dataframes have same number of rows

In [23]:
model1 = LogisticRegression()
model1.fit(X_train, Y_train_Sex)

c:\Users\victo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\victo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [24]:
model2 = LogisticRegression()
model2.fit(X_train, Y_train_ADHD)

c:\Users\victo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\victo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Bundling models together for multiple targets

In [48]:
X_train

,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,0throw_10thcolumn,...,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial
0,0.222930,0.527903,0.429966,0.060457,0.566489,0.315342,0.508408,-0.078290,0.525692,0.470063,...,29.0,0.0,8.0,1.0,4.0,5.0,4.0,4.0,3.0,9.0
1,-0.116833,0.458408,0.260703,0.639031,0.769337,0.442528,0.637110,0.192010,0.520379,0.378557,...,29.0,8.0,26.0,4.0,17.0,8.0,9.0,9.0,5.0,8.0
2,0.199688,0.752714,0.658283,0.575096,0.692867,0.645789,0.522750,0.412188,0.530843,0.259596,...,24.0,0.0,7.0,2.0,4.0,1.0,4.0,3.0,1.0,10.0
3,0.227321,0.613268,0.621447,0.562673,0.736709,0.589813,0.266676,0.359668,0.300771,0.331445,...,23.0,3.0,14.0,3.0,6.0,3.0,3.0,8.0,5.0,6.0
4,0.337316,0.823123,0.733585,0.418883,0.771445,0.653699,0.693189,0.345408,0.729822,0.357762,...,18.0,4.0,17.0,4.0,13.0,4.0,9.0,4.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,0.265284,0.551726,0.628318,0.647700,0.710757,0.132334,0.326207,0.354319,0.531170,0.340388,...,20.0,0.0,5.0,3.0,2.0,0.0,2.0,3.0,0.0,8.0
1155,-0.018377,0.576689,0.527451,0.327463,0.586868,0.573689,0.300544,0.301103,0.659840,0.434427,...,29.0,3.0,16.0,1.0,12.0,5.0,9.0,4.0,3.0,8.0
1156,0.227028,0.405659,0.023545,-0.093085,-0.068960,0.647574,0.762552,0.465109,0.199337,-0.357953,...,25.0,2.0,14.0,2.0,12.0,5.0,10.0,2.0,0.0,8.0
1157,0.189849,0.752876,0.842463,0.817037,0.820196,0.792950,0.650929,0.488504,0.580207,0.162963,...,21.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,9.0


In [49]:
Y_train_all

,ADHD_Outcome,Sex_F
0,1,0
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
1154,0,1
1155,1,0
1156,1,0
1157,0,1


In [47]:
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
# Wrap Logistic Regression in a MultiOutputClassifier
multi_target_model = MultiOutputClassifier(LogisticRegression())
multi_target_model.fit(X_train, Y_train_all)  # y_train_all has two columns

c:\Users\victo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\victo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

MultiOutputClassifier(estimator=LogisticRegression())

In [25]:
X_train.shape

(1159, 19917)

In [26]:
Y_train_ADHD.shape

(1159, 1)

## X Test

In [39]:
file_path_testQ = "../../data/raw/test/TEST_QUANTITATIVE_METADATA.csv"
test_Quant = pd.read_csv(file_path_testQ)
test_Quant = test_Quant.drop(columns=['MRI_Track_Age_at_Scan'])
test_Quant = test_Quant.dropna()
test_Quant

,participant_id,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial
0,Cfwaf5FX7jWK,60.03,14.0,5.0,16.0,41.0,19.0,11.0,26.0,2.0,12.0,3.0,9.0,2.0,7.0,3.0,0.0,8.0
1,vhGrzmvA3Hjq,86.71,12.0,3.0,13.0,43.0,18.0,15.0,28.0,2.0,16.0,8.0,5.0,7.0,3.0,11.0,3.0,9.0
2,ULliyEXjy4OV,26.68,13.0,3.0,14.0,36.0,16.0,14.0,25.0,1.0,7.0,1.0,6.0,1.0,5.0,1.0,0.0,9.0
3,LZfeAb1xMtql,93.38,13.0,3.0,19.0,41.0,17.0,18.0,27.0,4.0,15.0,4.0,10.0,8.0,6.0,5.0,1.0,6.0
4,EnFOUv0YK1RG,-93.38,14.0,3.0,13.0,42.0,19.0,16.0,28.0,2.0,18.0,6.0,12.0,5.0,10.0,6.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,UadZfjdEg7eG,86.71,14.0,3.0,17.0,41.0,17.0,11.0,25.0,1.0,18.0,7.0,7.0,5.0,6.0,11.0,4.0,7.0
300,IUEHiLmQAqCi,73.37,14.0,5.0,12.0,38.0,22.0,9.0,29.0,2.0,16.0,2.0,11.0,5.0,9.0,5.0,3.0,8.0
301,cRySmCadYFRO,87.84,13.0,3.0,14.0,42.0,15.0,10.0,28.0,1.0,11.0,4.0,4.0,4.0,3.0,7.0,3.0,10.0
302,E3MvDUtJadc5,46.76,14.0,3.0,16.0,43.0,18.0,12.0,22.0,5.0,21.0,2.0,10.0,6.0,5.0,11.0,9.0,0.0


In [40]:
file_path_testFM = "../../data/raw/test/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv"
test_FM = pd.read_csv(file_path_testFM)
test_FM

,participant_id,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
0,Cfwaf5FX7jWK,0.548480,0.713607,0.557319,0.524369,0.693364,0.770032,0.724406,0.390118,0.547912,...,0.080423,-0.054581,-0.088163,-0.028574,0.444847,0.350149,-0.012601,0.665750,0.560565,0.555732
1,vhGrzmvA3Hjq,0.427740,0.363022,0.402862,0.363003,0.534558,0.345347,0.409471,0.303328,0.402515,...,0.198009,-0.000724,0.083122,0.033043,0.687497,0.306229,0.717485,0.461809,0.559632,0.350027
2,ULliyEXjy4OV,0.139572,0.390106,-0.087041,0.196852,0.088148,0.023843,0.381782,0.068979,0.377488,...,0.051319,0.023630,-0.056819,0.117396,0.576086,0.517831,0.527044,0.605038,0.609856,0.750987
3,LZfeAb1xMtql,0.133561,0.778326,0.416355,0.471840,0.568460,0.633660,0.501113,0.345461,0.467943,...,0.046183,-0.238962,0.121868,-0.260970,0.646818,0.594902,0.608156,0.595459,0.683189,0.542296
4,EnFOUv0YK1RG,0.126699,0.575446,0.509422,0.363193,0.427544,0.449924,0.451796,0.223927,0.298248,...,0.315734,0.002234,0.290791,0.344149,0.480214,0.539824,0.447322,0.293088,0.148529,0.539823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,UadZfjdEg7eG,0.029721,0.596742,0.361898,0.292066,0.553130,0.476284,0.543970,0.372932,0.595183,...,0.021524,-0.162832,0.105546,0.028975,0.576180,0.378365,0.399277,0.556115,0.490833,0.568742
300,IUEHiLmQAqCi,0.025985,0.347840,0.304804,0.463090,0.305321,0.267397,0.468712,0.155677,0.298155,...,0.044974,-0.078047,-0.118271,0.035741,0.640073,0.515096,0.206462,0.536001,0.318518,0.305596
301,cRySmCadYFRO,0.105385,0.443624,-0.019777,0.435582,0.376492,0.619322,0.430177,0.567600,0.399820,...,0.145582,0.004455,0.134574,-0.152951,0.782938,0.737555,0.593972,0.734463,0.680979,0.678973
302,E3MvDUtJadc5,0.345641,0.758986,0.748153,0.442141,0.571924,0.485011,0.530513,0.137672,0.467144,...,0.036169,-0.345563,-0.165280,-0.347035,0.657024,0.590523,0.094565,0.727339,0.429694,0.530238


In [41]:
X_test_ppl = pd.merge(test_FM, test_Quant, on = 'participant_id')
X_test_ppl

,participant_id,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,...,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial
0,Cfwaf5FX7jWK,0.548480,0.713607,0.557319,0.524369,0.693364,0.770032,0.724406,0.390118,0.547912,...,26.0,2.0,12.0,3.0,9.0,2.0,7.0,3.0,0.0,8.0
1,vhGrzmvA3Hjq,0.427740,0.363022,0.402862,0.363003,0.534558,0.345347,0.409471,0.303328,0.402515,...,28.0,2.0,16.0,8.0,5.0,7.0,3.0,11.0,3.0,9.0
2,ULliyEXjy4OV,0.139572,0.390106,-0.087041,0.196852,0.088148,0.023843,0.381782,0.068979,0.377488,...,25.0,1.0,7.0,1.0,6.0,1.0,5.0,1.0,0.0,9.0
3,LZfeAb1xMtql,0.133561,0.778326,0.416355,0.471840,0.568460,0.633660,0.501113,0.345461,0.467943,...,27.0,4.0,15.0,4.0,10.0,8.0,6.0,5.0,1.0,6.0
4,EnFOUv0YK1RG,0.126699,0.575446,0.509422,0.363193,0.427544,0.449924,0.451796,0.223927,0.298248,...,28.0,2.0,18.0,6.0,12.0,5.0,10.0,6.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,UadZfjdEg7eG,0.029721,0.596742,0.361898,0.292066,0.553130,0.476284,0.543970,0.372932,0.595183,...,25.0,1.0,18.0,7.0,7.0,5.0,6.0,11.0,4.0,7.0
255,IUEHiLmQAqCi,0.025985,0.347840,0.304804,0.463090,0.305321,0.267397,0.468712,0.155677,0.298155,...,29.0,2.0,16.0,2.0,11.0,5.0,9.0,5.0,3.0,8.0
256,cRySmCadYFRO,0.105385,0.443624,-0.019777,0.435582,0.376492,0.619322,0.430177,0.567600,0.399820,...,28.0,1.0,11.0,4.0,4.0,4.0,3.0,7.0,3.0,10.0
257,E3MvDUtJadc5,0.345641,0.758986,0.748153,0.442141,0.571924,0.485011,0.530513,0.137672,0.467144,...,22.0,5.0,21.0,2.0,10.0,6.0,5.0,11.0,9.0,0.0


In [42]:
X_test = X_test_ppl.drop(columns=['participant_id'])

In [43]:
X_test.shape

(259, 19917)

In [44]:
X_train.shape

(1159, 19917)

In [33]:
# X_test = pd.merge(test_FM, test_Quant, on = 'participant_id')
# X_test = pd.merge(participant_id,X_test , on = 'participant_id')

In [37]:
X_test.isnull().sum()

0throw_1thcolumn              0
0throw_2thcolumn              0
0throw_3thcolumn              0
0throw_4thcolumn              0
0throw_5thcolumn              0
                             ..
SDQ_SDQ_Generating_Impact    30
SDQ_SDQ_Hyperactivity        30
SDQ_SDQ_Internalizing        30
SDQ_SDQ_Peer_Problems        30
SDQ_SDQ_Prosocial            30
Length: 19917, dtype: int64

### predicting

In [ ]:
y_pred = multi_target_model.predict(X_test)
multi_pred = multi_target_model.predict(X_test)

In [ ]:
y_pred = model2.predict(X_test) #ADHD??

In [ ]:
# how to submit (predicting from testing data and creating a csv file of your model's predictions)
# X_test is whatever you named your merged data from the testing datasets
# 'submission.csv' will be the name of the csv that is outputted

# sub = pd.read_csv('data/SAMPLE_SUBMISSION.csv')
# y_pred = your_model_name_here.predict(X_test)
# sub['ADHD_Outcome'] = y_pred[:,0]
# sub['Sex_F'] = y_pred[:,1]
# sub.to_csv('submission.csv', index=False)

In [57]:
X_test.shape

(259, 19917)

In [ ]:
y

In [56]:
sub = pd.read_csv('../../data/SAMPLE_SUBMISSION.csv')
multi_pred = multi_target_model.predict(X_test)
sub['ADHD_Outcome'] = multi_pred[:,0]
# sub['Sex_F'] = y_pred[:,1]
# sub.to_csv('submission.csv', index=False)

ValueError: Length of values (259) does not match length of index (304)

In [46]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Example for one target
print("Accuracy:", accuracy_score(Y_train_ADHD[:, 0], y_pred[:, 0]))
print("Precision:", precision_score(Y_train_ADHD[:, 0], y_pred[:, 0]))
print("Recall:", recall_score(Y_train_ADHD[:, 0], y_pred[:, 0]))
print("F1 Score:", f1_score(Y_train_ADHD[:, 0], y_pred[:, 0]))

InvalidIndexError: (slice(None, None, None), 0)